In [ ]:
#step 1

# Dependencies
import pandas as pd
import numpy as np
import csv

In [ ]:
# read the provided CSV data into DataFrames
stations="hawaii_stations.csv"
weather="hawaii_measurements.csv"
stations_data=pd.read_csv(stations,encoding="iso-8859-1")
weather_data=pd.read_csv(weather,encoding="iso-8859-1")
stations_df=pd.DataFrame(stations_data)
weather_df=pd.DataFrame(weather_data)


In [ ]:
# look at data to see missing data points.  looks like NaN's in 'prcp' column.
weather_df.count()

In [ ]:
# Quick look at the data
weather_df.head()

In [ ]:
# look at the numbers
weather_df.describe()

In [ ]:
# look at data to see missing data points.
stations_df.count()

In [ ]:
# full look at the data, appears to be in good shape.
stations_df

In [ ]:
# a pivot table to look at the average rain and temp numbers per station
# will help be evaluate the impact of the NaN's.

nan_eval=weather_df
nan_eval=pd.pivot_table(nan_eval,index=['station'], values=['prcp','tobs'])
nan_eval

In [ ]:
# pivot table to look at the NaN impact of rain volumn on each station.  Since it ranges from 0
# to just over 50% I do not want to drop rows without rain values since I will also be losing
# the temperature (significant impact). From prior pivot table, it appears that if I replace the
# NaN's with 0's I will have less data impact than dropping all of the rows with NaN's.

test=weather_df
test['NaN Count']=test['prcp'].isnull()
test['Count']=1
test=pd.pivot_table(test,index=["station"], values=['Count','NaN Count'],aggfunc=np.sum)
test['percentage']=test['NaN Count']/test['Count']*100
test

In [ ]:
# Look at the two files and make sure the weather stations match up.

stat_test1=stations_df['station']
stat_test2=weather_df['station'].unique()
clean= (stat_test1==stat_test2)
clean

In [ ]:
# Last check is to review the rain and temp data for reasonability 

minrain = weather_df['prcp'].min()
maxrain = weather_df['prcp'].max()
maxtemp = weather_df['tobs'].max()
mintemp = weather_df['tobs'].min()
print('maxrain : ', maxrain, ' | minrain : ',minrain, ' | maxtemp : ', maxtemp, ' | mintemp : ',mintemp)

In [ ]:
# replace NaN's with number 0 and save the clean data.  The rain numbers are small (first pivottable)
# and I don't want to trash the temp data, so rather than dropping all the good temp data I'm putting
#  zero in the NaN place since it looks like it won't impact the data as much as the loss of temp.

clean_stations_df=pd.DataFrame(stations_data)
clean_weather_df=pd.DataFrame(weather_data)
clean_weather_df.drop('NaN Count', axis=1, inplace=True)
clean_weather_df.drop('Count', axis=1, inplace=True)
clean_weather_df = clean_weather_df.fillna(0)
clean_weather_df.to_csv("clean_hawaii_measurements.csv", index = False)
clean_stations_df.to_csv("clean_hawaii_stations.csv", index = False)

In [ ]:
clean_weather_df

In [ ]:
#Step 2

# Import SQLAlchemy and other dependencies 
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
import pandas as pd


In [ ]:
# Create an engine to a database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [ ]:
# Use `declarative_base` from SQLAlchemy to model the tables as an ORM class
Base = declarative_base()

In [ ]:
# Create Measurement Class
# ----------------------------------
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    date = Column(String(15))
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [ ]:
# Create Station class
# ---------------------------------------
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    name = Column(String(45))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [ ]:
# Using `create_all`, creates the tables in the database
Base.metadata.create_all(engine)

In [ ]:
# Load the cleaned csv file into a pandas dataframe
df_of_measurements = pd.read_csv('clean_hawaii_measurements.csv')
df_of_stations = pd.read_csv('clean_hawaii_stations.csv')

In [ ]:
# Use Orient='records' to create a list of data to write
weather_data = df_of_measurements.to_dict(orient='records')
station_data = df_of_stations.to_dict(orient='records')
weather_data[0]

In [ ]:
# Use MetaData from SQLAlchemy to reflect the tables

metadata = MetaData(bind=engine)
metadata.reflect()

In [ ]:
# Save the reference to the `demographics` table as a variable called `table`

weather_table = sqlalchemy.Table('measurements', metadata, autoload=True)
station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [ ]:
# Use `table.delete()` to remove any existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.

conn.execute(weather_table.delete())
conn.execute(station_table.delete())

In [ ]:
# Use `table.insert()` to insert the data into the table

conn.execute(weather_table.insert(), weather_data)
conn.execute(station_table.insert(), station_data)

In [ ]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurements limit 5").fetchall()

In [ ]:
conn.execute("select * from station").fetchall()


In [ ]:
#Step 3

# Import SQLAlchemy and other dependencies 
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import datetime as dt
import pandas as pd
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
from sqlalchemy import Column, Integer, String, Float, Text, ForeignKey
import numpy as np

In [ ]:
# Create engine using the `hawaii.sqlite` database file created in database_engineering steps

engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# Declare a Base using `automap_base()`

Base = automap_base()



In [ ]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)



In [ ]:
# Print all of the classes mapped to the Base

Base.classes.keys()

In [ ]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

In [ ]:
# Use `engine.execute` to select and display the first 10 rows from the table

engine.execute('SELECT * FROM measurements LIMIT 10').fetchall()

In [ ]:
# Reflect Database into ORM class
Station = Base.classes.station
Measurements = Base.classes.measurements

In [ ]:
# Start a session to query the database
session = Session(engine)

In [ ]:
# Grabs the last date entry in the data table
last_date = session.query(Measurements.date).order_by(Measurements.date.desc()).first()
print(last_date)

In [ ]:
# to get the last 12 months of data, last date - 365
last_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(last_year)

In [ ]:
# query to pull the last year of precipitation data
rain = session.query(Measurements.date, Measurements.prcp).\
    filter(Measurements.date > last_year).\
    order_by(Measurements.date).all()

In [ ]:
# Load the data from the query into a dataframe

rain_df = pd.DataFrame(rain)
rain_df.head()

In [ ]:
# reset the index to the date

rain_df.set_index('date').head()

In [ ]:
# plot the dataframe
rain_df.plot('date', 'prcp')
plt.xlabel("Date")
plt.ylabel("Rain in Inches")
plt.title("Precipitation Analysis (8/24/16 to 8/23/17)")
plt.legend(["Precipitation"])
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to print the summary statistics for the precipitation data.
rain_df.describe()

In [ ]:
# Count the number of stations in the Measurement table
locations = session.query(Measurements).group_by(Measurements.station).count()
print("There are {} stations.".format(locations))

In [ ]:
# find the station with the most temperature observations, then list them all in descending order
busy_station = session.query(Measurements.station, func.count(Measurements.tobs)).group_by(Measurements.station).\
               order_by(func.count(Measurements.tobs).desc()).all()

busiest = busy_station[0][0]    
print("The busiest Station was",busiest,"with",busy_station[0][1],"weather observations.")
print()
print("Here are all of the Stations (in descending order) with their number of observations:")
for station, count in busy_station:
    print("Station",station,"had",count, "weather observations.")

In [ ]:
# query to pull the last year of precipitation data for the busiest station
temperature = session.query(Measurements.station, Measurements.date, Measurements.tobs).\
    filter(Measurements.station == busiest).\
    filter(Measurements.date > last_year).\
    order_by(Measurements.date).all()

In [ ]:
# plot the temperature data in a histogram with 12 bins
temp_df=pd.DataFrame(temperature)
plt.hist(temp_df['tobs'],12)
plt.xlabel("Recorded Temperature")
plt.ylabel("Number of Recorded Observations")
plt.title("Station Analysis (8/24/16 to 8/23/17) for Station " + busiest)
plt.show()

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurements.tobs), func.avg(Measurements.tobs), func.max(Measurements.tobs)).\
        filter(Measurements.date >= start_date).filter(Measurements.date <= end_date).all()
temp_range = (calc_temps('2012-02-28', '2012-03-05'))
print(temp_range)

In [ ]:
# calculate the min/max/avg from last year that matches my trip date
trip_arrive = dt.date(2018, 4, 1)
trip_leave = dt.date(2018, 4, 15)
last_year = dt.timedelta(days=365)
temp_avg_lst_year = (calc_temps((trip_arrive-last_year), (trip_leave-last_year)))
print(temp_avg_lst_year)

In [ ]:
# plotting the data as a boxplot

fig, ax = plt.subplots()
x = range(len(temp_avg_lst_year))
ax.boxplot(temp_avg_lst_year, patch_artist=True)
ax.set_title('Trip Average Temperature From Prior Year')
ax.set_ylabel("Temperature")
ax.set_xlabel("Trip")
fig.tight_layout()
plt.show()

In [ ]:
# plot the calculated min/max/avg from last year for the trip dates as a bar chart

p2p = temp_avg_lst_year[0][2]-temp_avg_lst_year[0][0]
avgtemp = temp_avg_lst_year[0][1]
min_temp = temp_avg_lst_year[0][0]
max_temp = temp_avg_lst_year[0][2]

fig, ax = plt.subplots()
bar_chart = ax.bar(1 , avgtemp, color= 'red', yerr=p2p)
ax.set_xlabel("Trip")
ax.set_ylabel("Temperature")
ax.set_title("Trip Average Temperature From Prior Year")

def autolabels(rects):
    for rect in rects:
        h=rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., .6*h,'%.2f' % float(h) ,ha='center', va='bottom', fontsize=10)

        
    # label the bars 
autolabels(bar_chart)        
plt.ylim(0, 100)
plt.xlim(0,2)
fig.tight_layout()
plt.show()


In [ ]:
# set the trip variables of arrive and leave for prior year
lst_year_arrive = trip_arrive - last_year
lst_year_leave = trip_leave - last_year
print(lst_year_arrive)
print(lst_year_leave)

In [ ]:
# query to pull the rain fall from last year matching this years trip
vacation_data = session.query(Measurements.station, Measurements.date, Measurements.prcp, Measurements.tobs).\
    filter(Measurements.date >= lst_year_arrive).\
    filter(Measurements.date <= lst_year_leave).\
    order_by(Measurements.station).all()

In [ ]:
# load the query results into a dataframe and pivot on station
vacation_data_df=pd.DataFrame(vacation_data)

rain_per_station = pd.pivot_table(vacation_data_df,index=['station'],values=['prcp'], aggfunc=sum)
rain_per_station

In [ ]:
#Step 4

# import dependencies 
import datetime as dt
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify


In [ ]:
# Database Setup

engine = create_engine("sqlite:///Hawaii.sqlite")
# reflect the database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# Save reference to the table
Station = Base.classes.station
Measurements = Base.classes.measurements

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [ ]:
# Flask Setup
app = Flask(__name__)


In [ ]:
# Flask Routes

@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"- List of prior year rain totals from all stations<br/>"
        f"<br/>"
        f"/api/v1.0/stations<br/>"
        f"- List of Station numbers and names<br/>"
        f"<br/>"
        f"/api/v1.0/tobs<br/>"
        f"- List of prior year temperatures from all stations<br/>"
        f"<br/>"
        f"/api/v1.0/start<br/>"
        f"- When given the start date (YYYY-MM-DD), calculates the MIN/AVG/MAX temperature for all dates greater than and equal to the start date<br/>"
        f"<br/>"
        f"/api/v1.0/start/end<br/>"
        f"- When given the start and the end date (YYYY-MM-DD), calculate the MIN/AVG/MAX temperature for dates between the start and end date inclusive<br/>"

    )

In [ ]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    """Return a list of rain fall for prior year"""
#    * Query for the dates and precipitation observations from the last year.
#           * Convert the query results to a Dictionary using `date` as the key and `prcp` as the value.
#           * Return the json representation of your dictionary.
    last_date = session.query(Measurements.date).order_by(Measurements.date.desc()).first()
    last_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    rain = session.query(Measurements.date, Measurements.prcp).\
        filter(Measurements.date > last_year).\
        order_by(Measurements.date).all()

In [ ]:
# Create a list of dicts with `date` and `prcp` as the keys and values
    rain_totals = []
    for result in rain:
        row = {}
        row["date"] = rain[0]
        row["prcp"] = rain[1]
        rain_totals.append(row)

    return jsonify(rain_totals)

In [ ]:
@app.route("/api/v1.0/stations")
def stations():
    stations_query = session.query(Station.name, Station.station)
    stations = pd.read_sql(stations_query.statement, stations_query.session.bind)
    return jsonify(stations.to_dict())


In [ ]:
@app.route("/api/v1.0/tobs")
def tobs():
    """Return a list of temperatures for prior year"""
#    * Query for the dates and temperature observations from the last year.
#           * Convert the query results to a Dictionary using `date` as the key and `tobs` as the value.
#           * Return the json representation of your dictionary.
    last_date = session.query(Measurements.date).order_by(Measurements.date.desc()).first()
    last_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    temperature = session.query(Measurements.date, Measurements.tobs).\
        filter(Measurements.date > last_year).\
        order_by(Measurements.date).all()

In [ ]:
# Create a list of dicts with `date` and `tobs` as the keys and values
    temperature_totals = []
    for result in temperature:
        row = {}
        row["date"] = temperature[0]
        row["tobs"] = temperature[1]
        temperature_totals.append(row)

    return jsonify(temperature_totals)

In [ ]:
@app.route("/api/v1.0/<start>")
def trip1(start):

 # go back one year from start date and go to end of data for Min/Avg/Max temp   
    start_date= dt.datetime.strptime(start, '%Y-%m-%d')
    last_year = dt.timedelta(days=365)
    start = start_date-last_year
    end =  dt.date(2017, 8, 23)
    trip_data = session.query(func.min(Measurements.tobs), func.avg(Measurements.tobs), func.max(Measurements.tobs)).\
        filter(Measurements.date >= start).filter(Measurements.date <= end).all()
    trip = list(np.ravel(trip_data))
    return jsonify(trip)

In [ ]:
@app.route("/api/v1.0/<start>/<end>")
def trip2(start,end):

  # go back one year from start/end date and get Min/Avg/Max temp     
    start_date= dt.datetime.strptime(start, '%Y-%m-%d')
    end_date= dt.datetime.strptime(end,'%Y-%m-%d')
    last_year = dt.timedelta(days=365)
    start = start_date-last_year
    end = end_date-last_year
    trip_data = session.query(func.min(Measurements.tobs), func.avg(Measurements.tobs), func.max(Measurements.tobs)).\
        filter(Measurements.date >= start).filter(Measurements.date <= end).all()
    trip = list(np.ravel(trip_data))
    return jsonify(trip)

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)